# 7/18 ~ 7/25　までの研究成果

## Dosのみを学習したオートエンコーダを用いた不均衡データセットの精度向上


### 背景
- 全てのデータを学習したオートエンコーダではu2rを学習することすらできなかった．
- 巨大なDosのデータにu2rが飲み込まれてしまっている可能性がある．

### 目的
- 全てのデータを学習したオートエンコーダでは見込みのなかったu2rの精度の向上
- u2rデータがDosのデータのみを学習したオートエンコーダで再構成誤差が大きくなるかを検証する．

### 使用するデータセット
- KDD99

### 手法
### 結果
### 考察

### 1. 準備（読み込み，前処理）

ライブラリの読み込み，と各バージョンの出力

In [1]:
from utils_kdd99 import *
print_version()

python:      3.10.11
sklearn:     1.2.2
tensorflow:  2.12.0
keras:       2.12.0
numpy:       1.23.5
pandas:      1.5.3


データを読み込み，説明変数と目的変数に分割する．

In [2]:
data_x, data_y = load_data(use_full_dataset=True, standard_scale=True, verbose=0, )

目的変数の種類と，データの数

In [3]:
data_y.value_counts()

smurf              2807886
neptune            1072017
normal              972781
satan                15892
ipsweep              12481
portsweep            10413
nmap                  2316
back                  2203
warezclient           1020
teardrop               979
pod                    264
guess_passwd            53
buffer_overflow         30
land                    21
warezmaster             20
imap                    12
rootkit                 10
loadmodule               9
ftp_write                8
multihop                 7
phf                      4
perl                     3
spy                      2
Name: true_label, dtype: int64

4つのクラスラベルに変換する．

In [4]:
data_y = data_y.map(lambda x: attack_label_class[x])

変換後の目的変数の種類と，データの数

In [5]:
data_y.value_counts()

dos       3883370
normal     972781
probe       41102
r2l          1126
u2r            52
Name: true_label, dtype: int64

クラスラベルを数値に変換

In [6]:
data_y = data_y.map(lambda x: correspondences[x])

k分割交差検証
- 訓練データと検証データの比率は2:1
- 訓練データと検証データの正解ラベルの各種類の比率は一定（stratify=True）

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.33, random_state=RANDOM_SEED, stratify=data_y)

訓練データと検証データのサイズ確認

In [8]:
print(f"x_train: {x_train.shape}, x_test: {x_test.shape}")


x_train: (3281948, 38), x_test: (1616483, 38)


### Dosのみを学習したオートエンコーダの作成
- 隠れ層の次元数(38->10->5->10->38)
- 活性化関数：ReLU
- 最適化関数：adam
- 損失関数：平均二乗誤差
- エポック数：1
- バッチサイズ：32

In [9]:
ae_model = keras.Sequential([
    Dense(units=10, activation='relu', input_dim=38, name='encoder1'),
    Dense(units=5, activation='relu', name='encoder2'),
    Dense(units=10, activation='relu'),
    Dense(units=38, activation='relu'),
])
ae_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
ae_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder1 (Dense)            (None, 10)                390       
                                                                 
 encoder2 (Dense)            (None, 5)                 55        
                                                                 
 dense (Dense)               (None, 10)                60        
                                                                 
 dense_1 (Dense)             (None, 38)                418       
                                                                 
Total params: 923
Trainable params: 923
Non-trainable params: 0
_________________________________________________________________


Dosのみのデータセットを作成

In [10]:
dos_x_train = x_train[y_train == correspondences['dos']]

オートエンコーダの学習

In [11]:
ae_model.fit(dos_x_train, dos_x_train,
          epochs=1, # データセットを使って学習する回数
        batch_size=32,
        shuffle=True,
        verbose=1,
        use_multiprocessing=True
          )

2023-07-25 07:24:21.983575: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


81309/81309 [==============================] - 38s 466us/step - loss: 0.2240 - accuracy: 0.8130


エンコーダー部分を抜き取る

In [12]:
encoder = keras.Sequential([ae_model.get_layer('encoder1'),
                            ae_model.get_layer('encoder2')])


Dosで学習したエンコーダによって出力された特徴量を命名

In [13]:
dos_columns = list(map(lambda x: 'dos' + str(x), range(5)))
dos_columns

['dos0', 'dos1', 'dos2', 'dos3', 'dos4']

特徴量抽出

In [14]:
x_train_encoded = pd.DataFrame(data=encoder.predict(x_train), index=x_train.index, columns=dos_columns)
x_test_encoded = pd.DataFrame(data=encoder.predict(x_test), index=x_test.index, columns=dos_columns)
x_train_encoded.head()


50516/50516 [==============================] - 12s 229us/step


,dos0,dos1,dos2,dos3,dos4
1537115,4.783251,2.364501,1.397150,2.007124,6.952895
4657510,20.224909,3.225566,0.000000,9.360068,0.000000
1894345,4.780805,2.360362,1.392731,2.009751,6.949811
1649777,4.783251,2.364501,1.397150,2.007124,6.952895
1634756,4.783251,2.364501,1.397150,2.007124,6.952895


Dosで学習した特徴量を元の特徴量にマージする

In [15]:
x_train_new_feature = x_train.merge(x_train_encoded, right_index=True, left_index=True)
x_test_new_feature = x_test.merge(x_test_encoded, right_index=True, left_index=True)
x_train_new_feature.head()


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,dos0,dos1,dos2,dos3,dos4
1537115,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,4.783251,2.364501,1.397150,2.007124,6.952895
4657510,-0.066833,-0.001949,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,4.079245,4.079791,20.224909,3.225566,0.000000,9.360068,0.000000
1894345,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,4.780805,2.360362,1.392731,2.009751,6.949811
1649777,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,4.783251,2.364501,1.397150,2.007124,6.952895
1634756,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,4.783251,2.364501,1.397150,2.007124,6.952895


### LightGBMを用いた学習

In [16]:
lgb_train = lgb.Dataset(x_train_new_feature, y_train)
lgb_eval = lgb.Dataset(x_test_new_feature, y_test, reference=lgb_train)

# LightGBM parameters
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class': 5,
        'metric': {'multi_error'}, # 評価指標 : 誤り率(= 1-正答率)  another multi_logloss
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 1000, #1000回学習
        'verbose': 0
}

# モデルの学習
model = lgb.train(params, # パラメータ
            train_set=lgb_train, # トレーニングデータの指定
            valid_sets=lgb_eval, # 検証データの指定
            callbacks=[lgb.early_stopping(100)]
               )

# テストデータの予測 (クラス1の予測確率(クラス1である確率)を返す)
y_pred_prob = model.predict(x_test_new_feature)
# テストデータの予測 (予測クラス(0 or 1 or...)を返す)
y_pred = np.argmax(y_pred_prob, axis=1) # 一番大きい予測確率のクラスを予測クラスに
y_pred = pd.Series(y_pred)
y_pred.value_counts()


/Users/satun/anaconda3/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.138819 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

0    1281597
1     321107
2      13357
3        408
4         14
dtype: int64

In [17]:
print(classification_report(y_test, y_pred, target_names=correspondences.keys()))

              precision    recall  f1-score   support

         dos       1.00      1.00      1.00   1281513
      normal       1.00      1.00      1.00    321018
       probe       0.99      0.98      0.99     13564
         r2l       0.84      0.92      0.88       371
         u2r       0.00      0.00      0.00        17

    accuracy                           1.00   1616483
   macro avg       0.77      0.78      0.77   1616483
weighted avg       1.00      1.00      1.00   1616483



In [18]:
for key, confusion_matrix in zip(correspondences.keys(), multilabel_confusion_matrix(y_test, y_pred)):
    print(f"{key}    TP: {confusion_matrix[1][1]}, TN: {confusion_matrix[0][0]}, FP: {confusion_matrix[0][1]}, FN: {confusion_matrix[1][0]}")


dos    TP: 1281409, TN: 334782, FP: 188, FN: 104
normal    TP: 320829, TN: 1295187, FP: 278, FN: 189
probe    TP: 13282, TN: 1602844, FP: 75, FN: 282
r2l    TP: 343, TN: 1616047, FP: 65, FN: 28
u2r    TP: 0, TN: 1616452, FP: 14, FN: 17


LightGBMモデルに学習データを予測させ，十分に学習できているか検証する

In [19]:
y_pred_prob: np.ndarray = model.predict(x_train_new_feature)
# テストデータの予測 (予測クラス(0 or 1 or...)を返す)
y_pred = np.argmax(y_pred_prob, axis=1) # 一番大きい予測確率のクラスを予測クラスに
y_pred = pd.Series(y_pred)
y_pred.value_counts()


0    2602095
1     651978
2      27083
3        782
4         10
dtype: int64

In [20]:
print(classification_report(y_train, y_pred, target_names=correspondences.keys()))


              precision    recall  f1-score   support

         dos       1.00      1.00      1.00   2601857
      normal       1.00      1.00      1.00    651763
       probe       1.00      0.98      0.99     27538
         r2l       0.90      0.94      0.92       755
         u2r       0.00      0.00      0.00        35

    accuracy                           1.00   3281948
   macro avg       0.78      0.78      0.78   3281948
weighted avg       1.00      1.00      1.00   3281948



そもそも，u2rに関しては，学習すらできていない．

In [21]:
params['is_unbalance'] = True
# モデルの学習
model = lgb.train(params, # パラメータ
            train_set=lgb_train, # トレーニングデータの指定
            valid_sets=lgb_eval, # 検証データの指定
            callbacks=[lgb.early_stopping(100)]
               )

# テストデータの予測 (クラス1の予測確率(クラス1である確率)を返す)
y_pred_prob = model.predict(x_train_new_feature)
# テストデータの予測 (予測クラス(0 or 1 or...)を返す)
y_pred = np.argmax(y_pred_prob, axis=1) # 一番大きい予測確率のクラスを予測クラスに
y_pred = pd.Series(y_pred)
y_pred.value_counts()

/Users/satun/anaconda3/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.163072 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

0    2602095
1     651978
2      27083
3        782
4         10
dtype: int64

In [22]:
print(classification_report(y_train, y_pred, target_names=correspondences.keys()))

              precision    recall  f1-score   support

         dos       1.00      1.00      1.00   2601857
      normal       1.00      1.00      1.00    651763
       probe       1.00      0.98      0.99     27538
         r2l       0.90      0.94      0.92       755
         u2r       0.00      0.00      0.00        35

    accuracy                           1.00   3281948
   macro avg       0.78      0.78      0.78   3281948
weighted avg       1.00      1.00      1.00   3281948



In [23]:
lgb.LGBMClassifier()


LGBMClassifier()